### Import Packages

In [78]:
import numpy as np
import torch.optim as optim
import torch
import torch.nn as nn

from utils.preprocess import getData
from networks.cnn import BasicCNN

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Load, Preprocess Data


In [58]:
X_train, y_train, X_valid, y_valid, X_test, y_test = getData()

### Shape of data

In [59]:
print('Training data shape: {}'.format(X_train.shape))
print('Valid data shape: {}'.format(X_valid.shape))
print('Test data shape: {}'.format(X_test.shape))

print('Training target shape: {}'.format(y_train.shape))
print('Valid target shape: {}'.format(y_valid.shape))
print('Test target shape: {}'.format(y_test.shape))


Training data shape: torch.Size([6960, 22, 250, 1])
Valid data shape: torch.Size([1500, 22, 250, 1])
Test data shape: torch.Size([1772, 22, 250, 1])
Training target shape: torch.Size([6960, 4])
Valid target shape: torch.Size([1500, 4])
Test target shape: torch.Size([1772, 4])


### Initialize Dataset and DataLoaders

In [60]:
batch_size=64
trainset = torch.utils.data.TensorDataset(X_train,y_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

# Shuffle is set to false for validation and test sets since no training is done on them, all we do is evaluate.
valset =  torch.utils.data.TensorDataset(X_valid, y_valid)
valloader = torch.utils.data.DataLoader(valset, batch_size=batch_size,
                                          shuffle=False, num_workers=2)

testset = torch.utils.data.TensorDataset(X_test, y_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                          shuffle=False, num_workers=2)

### Train a Convolutional Neural Network

In [79]:
# Initialize the network
network = BasicCNN()
print(network)

BasicCNN(
  (conv1): Conv2d(22, 25, kernel_size=(10, 1), stride=(1, 1), padding=same)
  (pool1): MaxPool2d(kernel_size=(3, 1), stride=1, padding=(1, 0), dilation=1, ceil_mode=False)
  (batchnorm1): BatchNorm2d(25, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(25, 50, kernel_size=(10, 1), stride=(1, 1), padding=same)
  (pool2): MaxPool2d(kernel_size=(3, 1), stride=1, padding=(1, 0), dilation=1, ceil_mode=False)
  (batchnorm2): BatchNorm2d(50, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv2d(50, 100, kernel_size=(10, 1), stride=(1, 1), padding=same)
  (pool3): MaxPool2d(kernel_size=(3, 1), stride=1, padding=(1, 0), dilation=1, ceil_mode=False)
  (batchnorm3): BatchNorm2d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv4): Conv2d(100, 200, kernel_size=(10, 1), stride=(1, 1), padding=same)
  (pool4): MaxPool2d(kernel_size=(3, 1), stride=1, padding=(1, 0), dilation=1, ceil_mode=False)
  (batchno

In [83]:
# Select loss criterion
criterion = nn.CrossEntropyLoss()

# create your optimizer
optimizer = optim.Adam(network.parameters(), lr=0.001)

# Train the network
num_epochs = 10

# Store the loss
train_accuracies = []
train_losses = []

for epoch in range(num_epochs): # loop over the dataset multiple times
    running_loss = 0.0
    correct = 0.0
    total = 0.0
    for i, data in enumerate(trainloader, 0):
        print('Iterating')
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.float()
        labels = labels.float()
  
        # forward pass
        outputs = network(inputs)
        
        loss = criterion(outputs, labels)
        

        # backward + optimize
        loss.backward() # backward to get gradient values
        
        optimizer.step() # does the update
    
        # zero the parameter gradients
        optimizer.zero_grad()
        
        # accumulate loss
        running_loss += loss.item()
        
        # Make prediction for batch
        _, predicted = outputs.max(1)
        
        # Store accuracy for batch
        # WE convert back from one-hot to integer for checking accuracy
        total += labels.size(0)
        correct += predicted.eq(torch.argmax(labels, dim=1)).sum().item()
        
    # Store accuracy,loss for epoch
    train_loss=running_loss/len(trainloader)
    train_accuracy=100.*correct/total
    
    train_accuracies.append(train_accuracy)
    train_losses.append(train_loss)
    
    # Display results
    print(f'Epoch: {epoch} | Train Loss: {train_loss} | Train Accuracy: {train_accuracy}')

Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating


Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating
Iterating


### Plot training curves

### Evaluate on Validation Set

In [86]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in valloader:
        inputs, labels = data
        
        inputs = inputs.float()
        labels = labels.float()
        
        # calculate outputs by running inputs through the network
        outputs = network(inputs)
        
        # the class with the highest energy is what we choose as prediction
        # Ww convert back from one-hot to integer for checking accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == torch.argmax(labels, dim=1)).sum().item()

print(f'Accuracy of the network on the {len(valloader.dataset)} validation examples: {100 * correct // total} %')

Accuracy of the network on the 1500 validation examples: 94 %


### Evaluate on Test Set

In [88]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        
        inputs = inputs.float()
        labels = labels.float()
        
        # calculate outputs by running inputs through the network
        outputs = network(inputs)
        
        # the class with the highest energy is what we choose as prediction
        # Ww convert back from one-hot to integer for checking accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == torch.argmax(labels, dim=1)).sum().item()

print(f'Accuracy of the network on the {len(testloader.dataset)} test examples: {100 * correct // total} %')

Accuracy of the network on the 1772 test examples: 53 %
